# Build the Neural Network 构建神经网络

神经网络由对数据执行操作的层/模块组成。 [torch.nn](https://pytorch.org/docs/stable/nn.html)命名空间提供了构建您自己的神经网络所需的所有构建块。 PyTorch 中的每个模块都是(nn.Module)[https://pytorch.org/docs/stable/generated/torch.nn.Module.html]的子类。神经网络本身就是一个模块，由其他模块（层）组成。这种嵌套结构允许轻松构建和管理复杂的架构。

在以下部分中，我们将构建一个神经网络来对 `FashionMNIST` 数据集中的图像进行分类。

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training 获取训练设备

我们希望能够在 GPU 或 MPS 等硬件加速器（如果可用）上训练我们的模型。让我们检查一下[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html)或(torch.backends.mps)[https://pytorch.org/docs/stable/notes/mps.html]是否可用，否则我们使用 CPU。

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


## Define the Class 定义类

我们通过子类化`nn.Module`来定义神经网络，并在`__init__`中初始化神经网络层。每个`nn.Module`子类都实现了`forward`方法中对输入数据的操作。

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

我们创建一个`NeuralNetwork`实例，并将其移动到`device`上，并打印其结构。

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


为了使用该模型，我们将输入数据传递给它。这将执行模型的forward以及一些[后台操作](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866)。不要直接调用`model.forward()`

在输入上调用模型会返回一个二维张量，其中 `dim=0` 对应于每个类的 10 个原始预测值的每个输出，`dim=1` 对应于每个输出的各个值。我们通过将预测概率传递给`nn.Softmax`模块的实例来获得预测概率。

In [5]:
X = torch.rand(3, 28, 28, device=device)
logits = model(X)
print(f"model logits:\n{logits}")
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

model logits:
tensor([[-0.0118,  0.0263,  0.0279,  0.1242,  0.0405,  0.0814, -0.0302, -0.1246,
         -0.0750,  0.0437],
        [-0.0075, -0.0157,  0.0388,  0.1584,  0.0617,  0.1117, -0.0177, -0.0422,
         -0.0661,  0.0711],
        [-0.0512,  0.0358,  0.0146,  0.1329,  0.1074,  0.0280, -0.0452, -0.0168,
         -0.0474,  0.0212]], device='mps:0', grad_fn=<LinearBackward0>)
Predicted class: tensor([3, 3, 3], device='mps:0')


## Model Layers 模型层

让我们分解 FashionMNIST 模型中的各个层。为了说明这一点，我们将采用 3 个大小为 28x28 的图像的小批量样本，并看看当我们将其传递到网络时会发生什么。

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### `nn.Flatten`

我们初始化`nn.Flatten`层，将每个 2D 28x28 图像转换为 784 个像素值的连续数组（维持小批量维度（在 `dim=0` 时））

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### `nn.Linear`

[线性层](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)是一个使用其存储的权重和偏差对输入应用线性变换的模块。

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### `nn.ReLU`

非线性激活是在模型的输入和输出之间创建复杂映射的原因。它们在线性变换后应用以引入非线性，帮助神经网络学习各种现象。

在此模型中，我们在线性层之间使用[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)，但还有其他激活可以在模型中引入非线性。

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0120, -0.2038,  0.0225, -0.1637, -0.2653,  0.2652,  0.1486, -0.0780,
         -0.5483,  0.2662, -0.1375,  0.1128, -0.0870,  0.4732, -0.6000, -0.5804,
          0.1340,  0.0281, -0.0238,  0.1907],
        [-0.4260, -0.4172,  0.0416,  0.2599, -0.6496, -0.0539,  0.1762,  0.1715,
         -0.7274,  0.2796, -0.1365,  0.0299, -0.0363,  0.2670, -0.3956, -0.6340,
         -0.3685,  0.2073,  0.1167,  0.3742],
        [-0.5934, -0.5214,  0.1683,  0.1446, -0.3364,  0.2474,  0.2864, -0.0302,
         -0.8655,  0.0860, -0.1926,  0.3460, -0.3002, -0.0481, -0.0352, -0.5657,
          0.1310, -0.0317,  0.0199,  0.2212]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0120, 0.0000, 0.0225, 0.0000, 0.0000, 0.2652, 0.1486, 0.0000, 0.0000,
         0.2662, 0.0000, 0.1128, 0.0000, 0.4732, 0.0000, 0.0000, 0.1340, 0.0281,
         0.0000, 0.1907],
        [0.0000, 0.0000, 0.0416, 0.2599, 0.0000, 0.0000, 0.1762, 0.1715, 0.0000,
         0.2796, 0.0000, 0.0299, 0.0000, 0.2670, 0.00

### `nn.Sequential`

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)是模块的有序容器。数据按照定义的相同顺序传递通过所有模块。您可以使用顺序容器来组合一个快速网络，例如`seq_modules`。

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

tensor([[-0.0263, -0.0073,  0.0043, -0.0495,  0.0560,  0.0378,  0.1985, -0.1308,
          0.1699, -0.0246],
        [-0.0225,  0.0528, -0.0500, -0.0111,  0.0664,  0.1160,  0.1757, -0.1098,
          0.1114,  0.0474],
        [ 0.0678,  0.0790,  0.0781,  0.0045,  0.1316,  0.0240,  0.1933, -0.1146,
          0.1610, -0.0028]], grad_fn=<AddmmBackward0>)


### `nn.Softmax`

神经网络的最后一个线性层返回`logits` - `[-infty, infty]` 中的原始值 - 被传递到`nn.Softmax`模块。 `Logits` 缩放为值 `[0, 1]`，表示模型对每个类别的预测概率。 dim参数指示值总和必须为 1 的维度。

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0948, 0.0966, 0.0977, 0.0926, 0.1029, 0.1011, 0.1187, 0.0854, 0.1153,
         0.0949],
        [0.0938, 0.1012, 0.0913, 0.0949, 0.1026, 0.1078, 0.1144, 0.0860, 0.1073,
         0.1006],
        [0.1002, 0.1013, 0.1012, 0.0941, 0.1068, 0.0959, 0.1136, 0.0835, 0.1100,
         0.0934]], grad_fn=<SoftmaxBackward0>)


## Model Parameters 模型参数

神经网络内的许多层都是参数化的，即具有在训练期间优化的相关权重和偏差。子类化`nn.Module`自动跟踪模型对象中定义的所有字段，并使用模型的`parameters()`或`named_parameters()`方法使所有参数可访问。

在此示例中，我们迭代每个参数，并打印其大小及其值的预览。

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0305, -0.0087, -0.0098,  ..., -0.0311,  0.0198,  0.0319],
        [ 0.0233,  0.0262, -0.0090,  ...,  0.0223,  0.0035, -0.0138]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0267, -0.0332], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0313, -0.0052, -0.0187,  ...,  0.0301, -0.0108, -0.0374],
        [-0.0043,  0.0028, -0.0131,  ...,  0.0198,  0.0303, -0.0259]],
       device='mps:0', grad_fn=<Slice